In [1]:
# Install dependencies (run this once)
!pip install ipywidgets pandas openpyxl --quiet

import pandas as pd
from ipywidgets import FileUpload, Dropdown, Checkbox, VBox, HBox, Button, Output
from IPython.display import display, clear_output, FileLink
import io
import os

# Widgets
upload = FileUpload(accept='.xlsx,.xls', multiple=False)
sheet_dropdown = Dropdown(description='Sheet:')
checkboxes_box = VBox()
generate_button = Button(description="✅ Generate CSV", button_style='success')
view_button = Button(description="🔍 View File", button_style='info')
clear_button = Button(description="🗑️ Clear Uploaded File", button_style='danger')

# Output areas
upload_output = Output()
sheet_output = Output()
checkbox_output = Output()
action_buttons = Output()
view_output = Output()
message_output = Output()

# Globals
excel_data = None
df = pd.DataFrame()
filtered_df = pd.DataFrame()
csv_filename = "selected_columns.csv"

# Reset everything
def reset_all():
    global excel_data, df, filtered_df, csv_filename
    excel_data = None
    df = pd.DataFrame()
    filtered_df = pd.DataFrame()
    csv_filename = "selected_columns.csv"
    upload.value.clear()
    sheet_dropdown.options = []
    checkboxes_box.children = []
    with upload_output: clear_output()
    with sheet_output: clear_output()
    with checkbox_output: clear_output()
    with action_buttons: clear_output()
    with view_output: clear_output()
    with message_output:
        clear_output()
        print("🗑️ Upload cleared. You can now upload a new file.")

# Handle file upload
def on_file_upload(change):
    global excel_data, csv_filename
    with upload_output:
        clear_output()
        if upload.value:
            file_name = list(upload.value.keys())[0]
            content = upload.value[file_name]['content']
            excel_data = pd.ExcelFile(io.BytesIO(content))

            # Get CSV filename from Excel filename
            base_name = os.path.splitext(file_name)[0]
            csv_filename = base_name + ".csv"

            sheet_dropdown.options = excel_data.sheet_names
            print(f"✅ File uploaded: {file_name}")
            display(sheet_dropdown)
            with action_buttons:
                clear_output()
                display(HBox([view_button, clear_button]))

# Handle sheet selection
def on_sheet_change(change):
    global df
    with sheet_output:
        clear_output()
        df = excel_data.parse(sheet_dropdown.value, header=0)
        checkboxes = [Checkbox(value=False, description=col) for col in df.columns]
        checkboxes_box.children = checkboxes
        display(checkboxes_box, generate_button)

# Generate CSV and show download link
def on_generate_clicked(b):
    global filtered_df, csv_filename
    selected_columns = [cb.description for cb in checkboxes_box.children if cb.value]
    with message_output:
        clear_output()
        if not selected_columns:
            print("⚠️ Please select at least one column.")
            return
        filtered_df = df[selected_columns]
        filtered_df.to_csv(csv_filename, index=False)
        print(f"✅ CSV generated successfully! Click below to download `{csv_filename}`:")
        display(FileLink(csv_filename, result_html_prefix="🔗 "))

# View sheet preview
def on_view_clicked(b):
    with view_output:
        clear_output()
        if excel_data is None:
            print("⚠️ No file uploaded.")
        else:
            print(f"📄 Preview of sheet: {sheet_dropdown.value}")
            display(excel_data.parse(sheet_dropdown.value).head(10))

# Attach callbacks
upload.observe(on_file_upload, names='value')
sheet_dropdown.observe(on_sheet_change, names='value')
generate_button.on_click(on_generate_clicked)
view_button.on_click(on_view_clicked)
clear_button.on_click(lambda b: reset_all())

# Display UI
print("📂 Upload your Excel file:")
display(upload, upload_output, sheet_output, checkbox_output, action_buttons, view_output, message_output)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.3 MB/s eta 0:00:00
📂 Upload your Excel file:


FileUpload(value={}, accept='.xlsx,.xls', description='Upload')

Output()

Output()

Output()

Output()

Output()

Output()